Example Demos provided by [github.com/nickochnack/OpenCV-Basics](github.com/nickochnack/OpenCV-Basics)

### Import Dependencies

In [1]:
import cv2
import os
from matplotlib import pyplot as plt

### Take a photo

In [3]:
def take_photo(filename): 
    cap = cv2.VideoCapture(2)
    ret, frame = cap.read()
    plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
    key = cv2.waitKey(1)
    if key == ord('s'):
            cv2.imwrite('{0}.jpg'.format(filename), frame)
            cap.release()
    if key == ord('q'):
          cap.release()

In [ ]:
take_photo("test")

### Take a Video

In [4]:
def takeVideo():
    # Connect to webcam
    cap = cv2.VideoCapture(2)
    # Loop through every frame until we close our webcam
    while cap.isOpened(): 
        ret, frame = cap.read()
        
        # Show image 
        cv2.imshow('Webcam', frame)
        
        # Checks whether q has been hit and stops the loop
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break

    # Releases the webcam
    cap.release()
    # Closes the frame
    cv2.destroyAllWindows()

In [10]:
takeVideo()

### Edge Detection Demo

In [12]:
def get_edges(): 
    cap = cv2.VideoCapture(2)
    
    while cap.isOpened(): 
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(frame, (3,3),0)
        canny = cv2.Canny(gray, threshold1=180,threshold2=200)
        # resized = cv2.resize(canny, (int(2190/2),int(2738/2)))
        cv2.imshow("Canny",canny)
        # Checks whether q has been hit and stops the loop
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break

    # Releases the webcam
    cap.release()
    # Closes the frame
    cv2.destroyAllWindows()

In [13]:
get_edges()